## CHAPTER 10
# Data Aggregation and Group Operations

In [109]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 10.1 How to Think About Group Operations

In [110]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", None, "b", "b", "a", None],
        "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1": np.random.standard_normal(7),
        "data2": np.random.standard_normal(7),
    }
)
df

,key1,key2,data1,data2
0,a,1,-1.457865,-0.165550
1,a,2,0.888363,-0.007215
2,None,1,1.307729,0.336994
3,b,2,-0.026734,0.503198
4,b,1,-0.580537,-0.246069
5,a,<NA>,1.112245,0.500007
6,None,1,0.673036,-0.275256


In [111]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [112]:
grouped.mean()

key1
a    0.180915
b   -0.303635
Name: data1, dtype: float64

In [113]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -1.457865
      2       0.888363
b     1      -0.580537
      2      -0.026734
Name: data1, dtype: float64

In [114]:
means.unstack()

key2,1,2
key1,,
a,-1.457865,0.888363
b,-0.580537,-0.026734


In [115]:
state = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = np.array([2005, 2005, 2006, 2005, 2006, 2005, 2006])
df["data1"].groupby([state, years]).mean()

CA  2005    1.000304
    2006    1.307729
OH  2005   -0.742299
    2006    0.046249
Name: data1, dtype: float64

In [116]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.180915,0.109081
b,1.5,-0.303635,0.128564


In [117]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,-0.014409,-0.087470
2,0.430815,0.247991


In [118]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -1.457865 -0.165550
     2     0.888363 -0.007215
b    1    -0.580537 -0.246069
     2    -0.026734  0.503198

In [119]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [120]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [121]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [122]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [123]:
for name , group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -1.457865 -0.165550
1    a     2  0.888363 -0.007215
5    a  <NA>  1.112245  0.500007
b
  key1  key2     data1     data2
3    b     2 -0.026734  0.503198
4    b     1 -0.580537 -0.246069


In [124]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1    data2
0    a     1 -1.457865 -0.16555
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2  0.888363 -0.007215
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1 -0.580537 -0.246069
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2 -0.026734  0.503198


In [125]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.026734,0.503198
4,b,1,-0.580537,-0.246069


In [126]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -1.457865 -0.165550
 1    a     2  0.888363 -0.007215
 5    a  <NA>  1.112245  0.500007,
 'b':   key1  key2     data1     data2
 3    b     2 -0.026734  0.503198
 4    b     1 -0.580537 -0.246069}

In [127]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -1.457865 -0.165550
1  0.888363 -0.007215
2  1.307729  0.336994
3 -0.026734  0.503198
4 -0.580537 -0.246069
5  1.112245  0.500007
6  0.673036 -0.275256
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\695880334.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


### Selecting a Column or Subset of Columns

In [128]:
df

,key1,key2,data1,data2
0,a,1,-1.457865,-0.165550
1,a,2,0.888363,-0.007215
2,None,1,1.307729,0.336994
3,b,2,-0.026734,0.503198
4,b,1,-0.580537,-0.246069
5,a,<NA>,1.112245,0.500007
6,None,1,0.673036,-0.275256


In [129]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [130]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [131]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1    -0.165550
     2    -0.007215
b    1    -0.246069
     2     0.503198

In [132]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [133]:
s_grouped.mean()

key1  key2
a     1      -0.165550
      2      -0.007215
b     1      -0.246069
      2       0.503198
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [134]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"],
)
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,2.344691,2.027155,0.698474,-2.031043,-1.298593
Steve,-0.461288,0.896020,0.729867,0.228195,0.155162
Wanda,0.401089,NaN,NaN,-0.712052,0.761907
Jill,-0.234867,-0.780744,0.927978,0.005941,0.919859
Trey,0.128879,-1.845514,-1.920855,-0.149703,-0.196671


In [135]:
mapping = {"a": "red", "b": "red", "c": "blue",
            "d": "blue", "e": "red", "f" : "orange"}


In [136]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,-1.332570,3.073253
Steve,0.958063,0.589894
Wanda,-0.712052,1.162996
Jill,0.933919,-0.095753
Trey,-2.070558,-1.913305


In [137]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [138]:
people.groupby(map_series, axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [139]:
people

,a,b,c,d,e
Joe,2.344691,2.027155,0.698474,-2.031043,-1.298593
Steve,-0.461288,0.896020,0.729867,0.228195,0.155162
Wanda,0.401089,NaN,NaN,-0.712052,0.761907
Jill,-0.234867,-0.780744,0.927978,0.005941,0.919859
Trey,0.128879,-1.845514,-1.920855,-0.149703,-0.196671


In [140]:
people.groupby(len).sum()  # group by length of the index value

,a,b,c,d,e
3,2.344691,2.027155,0.698474,-2.031043,-1.298593
4,-0.105988,-2.626257,-0.992878,-0.143762,0.723188
5,-0.060199,0.896020,0.729867,-0.483856,0.917069


In [141]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,2.344691,2.027155,0.698474,-2.031043,-1.298593
4,two,-0.234867,-1.845514,-1.920855,-0.149703,-0.196671
5,one,-0.461288,0.896020,0.729867,-0.712052,0.155162


### Grouping by Index Levels

In [142]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.682399  0.106141  1.028735  1.057930  1.433965
1      0.835810  0.608590 -0.252684  0.625924 -1.046489
2     -0.636907  0.335580 -0.882477 -1.346200  0.333536
3     -1.052945  0.193339 -0.072957 -0.597840  2.220777

In [143]:
hier_df.groupby(level="cty", axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [144]:
df

,key1,key2,data1,data2
0,a,1,-1.457865,-0.165550
1,a,2,0.888363,-0.007215
2,None,1,1.307729,0.336994
3,b,2,-0.026734,0.503198
4,b,1,-0.580537,-0.246069
5,a,<NA>,1.112245,0.500007
6,None,1,0.673036,-0.275256


In [145]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -1.457865
      1    0.888363
b     4   -0.580537
      3   -0.026734
Name: data1, dtype: float64

In [146]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.570110,0.665557
b,1,0.553803,0.749267


In [147]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.180915  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.303635  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     1.000304  1.112245   3.0  0.109081  0.347685 -0.165550 -0.086382   
b    -0.165185 -0.026734   2.0  0.128564  0.529812 -0.246069 -0.058752   

                                    
           50%       75%       max  
key1                                
a    -0.007215  0.246396  0.500007  
b     0.128564  0.315881  0.503198  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [148]:
# tips = sns.load_dataset("tips")
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [149]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [150]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [151]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [152]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [153]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [154]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [155]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\4198102554.py:2: FutureWarning: The provided callable <function var at 0x00000298C1D8DEE0> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [156]:
grouped.agg({"tip": np.max, "size": "sum"})

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\3453032214.py:1: FutureWarning: The provided callable <function max at 0x00000298C1D8D260> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip": np.max, "size": "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [157]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
            "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [158]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [159]:
tips.groupby(["day", "smoker"], as_index=False).mean(numeric_only=True) # Added numeric_only=True to avoid FutureWarning # শুধু numeric কলাম নাও

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply: General split-apply-combine

In [160]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [161]:
tips.groupby("smoker").apply(top)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2530541573.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [162]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\643835440.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [163]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [164]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [165]:
def f(group):
    return group.describe()
grouped.apply(f)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\3509319160.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(f)


total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suppressing the Group Keys

In [166]:
tips.groupby("smoker", group_keys=False).apply(top)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\3851957478.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Quantile and Bucket Analysis

In [167]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-0.196241,-0.818626
1,-0.501786,-1.270778
2,-1.088745,-0.323083
3,-0.778301,0.279098
4,-1.596378,-0.137822


In [168]:
quartiles = pd.cut(frame["data1"], 4)
quartiles[:10]

0     (-0.304, 1.267]
1    (-1.876, -0.304]
2    (-1.876, -0.304]
3    (-1.876, -0.304]
4    (-1.876, -0.304]
5    (-1.876, -0.304]
6     (-0.304, 1.267]
7     (-0.304, 1.267]
8     (-0.304, 1.267]
9    (-1.876, -0.304]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.454, -1.876] < (-1.876, -0.304] < (-0.304, 1.267] < (1.267, 2.839]]

In [169]:
def get_stats(group):
    return pd.DataFrame(
        {
            "min": group.min(),
            "max": group.max(),
            "count": group.count(),
            "mean": group.mean(),
            "std": group.std(),
        }
    )
grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2176047469.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


min       max  count      mean       std
data1                                                                
(-3.454, -1.876] data1 -3.447262 -1.894921     33 -2.352505  0.372952
                 data2 -3.554609  1.831077     33  0.119148  1.059489
(-1.876, -0.304] data1 -1.873258 -0.306430    368 -0.877333  0.411867
                 data2 -2.921248  2.651905    368  0.020312  1.065230
(-0.304, 1.267]  data1 -0.303452  1.264616    502  0.410939  0.424459
                 data2 -3.856400  4.026822    502 -0.091430  1.018602
(1.267, 2.839]   data1  1.300428  2.838905     97  1.780703  0.382667
                 data2 -2.701110  2.522862     97 -0.072379  1.022097

In [170]:
grouped.agg(['min', 'max', 'count', 'mean', 'std'])

data1                                         data2  \
                       min       max count      mean       std       min   
data1                                                                      
(-3.454, -1.876] -3.447262 -1.894921    33 -2.352505  0.372952 -3.554609   
(-1.876, -0.304] -1.873258 -0.306430   368 -0.877333  0.411867 -2.921248   
(-0.304, 1.267]  -0.303452  1.264616   502  0.410939  0.424459 -3.856400   
(1.267, 2.839]    1.300428  2.838905    97  1.780703  0.382667 -2.701110   

                                                      
                       max count      mean       std  
data1                                                 
(-3.454, -1.876]  1.831077    33  0.119148  1.059489  
(-1.876, -0.304]  2.651905   368  0.020312  1.065230  
(-0.304, 1.267]   4.026822   502 -0.091430  1.018602  
(1.267, 2.839]    2.522862    97 -0.072379  1.022097

In [171]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp[:10]

0    1
1    1
2    0
3    0
4    0
5    0
6    2
7    2
8    1
9    1
Name: data1, dtype: int64

In [172]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean       std
data1                                                     
0     data1 -3.447262 -0.696378    250 -1.305916  0.522150
      data2 -3.554609  2.651905    250  0.088593  1.050338
1     data1 -0.695706 -0.023107    250 -0.362651  0.190863
      data2 -3.239795  2.296980    250 -0.129048  1.085054
2     data1 -0.019051  0.661971    250  0.323666  0.187357
      data2 -3.856400  4.026822    250 -0.134979  0.999138
3     data1  0.662259  2.838905    250  1.259015  0.498442
      data2 -2.701110  2.522862    250  0.009387  1.003322

### Example: Filling Missing Values with Group-Specific Values

In [173]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1   -0.233513
2         NaN
3    1.076215
4         NaN
5    0.443208
dtype: float64

In [174]:
s.fillna(s.mean())

0    0.428637
1   -0.233513
2    0.428637
3    1.076215
4    0.428637
5    0.443208
dtype: float64

In [175]:
state = [
    "Ohio",
    "New York",
    "Vermont",
    "Florida",
    "Oregon",
    "Nevada",
    "California",
    "Idaho",
]

In [176]:
group_key = ["East"] * 4 + ["West"] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [177]:
data = pd.Series(np.random.standard_normal(8), index=state)
data

Ohio          0.042025
New York     -0.155996
Vermont       0.708591
Florida       1.511096
Oregon        2.270124
Nevada       -0.803716
California    0.679724
Idaho         1.047240
dtype: float64

In [178]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.042025
New York     -0.155996
Vermont            NaN
Florida       1.511096
Oregon        2.270124
Nevada             NaN
California    0.679724
Idaho              NaN
dtype: float64

In [179]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [180]:
data.groupby(group_key).mean()

East    0.465708
West    1.474924
dtype: float64

In [181]:
def fill_mean(group):
    return group.fillna(group.mean())

In [182]:
data.groupby(group_key).apply(fill_mean)

East  Ohio          0.042025
      New York     -0.155996
      Vermont       0.465708
      Florida       1.511096
West  Oregon        2.270124
      Nevada        1.474924
      California    0.679724
      Idaho         1.474924
dtype: float64

In [183]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [184]:
data.groupby(group_key).apply(fill_func)

East  Ohio          0.042025
      New York     -0.155996
      Vermont       0.500000
      Florida       1.511096
West  Oregon        2.270124
      Nevada       -1.000000
      California    0.679724
      Idaho        -1.000000
dtype: float64

In [185]:
data

Ohio          0.042025
New York     -0.155996
Vermont            NaN
Florida       1.511096
Oregon        2.270124
Nevada             NaN
California    0.679724
Idaho              NaN
dtype: float64

### Example: Random Sampling and Permutation

In [186]:
suits = ["H", "S", "C", "D"] # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "Q", "K"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [187]:
def drew(deck, n=5):
    return deck.sample(n)
drew(deck)

2C     2
KD    10
AH     1
8D     8
5S     5
dtype: int64

In [197]:
def get_suit(card):
    # last card is suit
    return card[-1]
deck.groupby(get_suit).apply(drew, n=2)

C  8C     8
   KC    10
D  8D     8
   9D     9
H  KH    10
   4H     4
S  5S     5
   KS    10
dtype: int64

In [213]:
deck.groupby(get_suit, group_keys=False).apply(drew, n=2)

AC      1
2C      2
2D      2
9D      9
AH      1
JH     10
KS     10
10S    10
dtype: int64

### Example: Group Weighted Average and Correlation

In [216]:
df = pd.DataFrame({
    "category": ["a", "a", "a", "a", "b", "b", "b", "b"],
    "data": np.random.standard_normal(8),
    "weights": np.random.uniform(size=8),
})
df

,category,data,weights
0,a,1.871967,0.064797
1,a,-0.138906,0.684184
2,a,0.664341,0.716495
3,a,1.452293,0.970224
4,b,0.056777,0.415363
5,b,0.469229,0.543986
6,b,-1.360300,0.182060
7,b,0.381594,0.605010


In [217]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])
grouped.apply(get_wavg)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\1926684416.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


category
a    0.784705
b    0.150049
dtype: float64

In [218]:
grouped.mean()

,data,weights
category,,
a,0.962424,0.608925
b,-0.113175,0.436605


In [219]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5472 entries, 1990-02-01 to 2011-10-14
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AA      5472 non-null   float64
 1   AAPL    5472 non-null   float64
 2   GE      5472 non-null   float64
 3   IBM     5472 non-null   float64
 4   JNJ     5472 non-null   float64
 5   MSFT    5472 non-null   float64
 6   PEP     5471 non-null   float64
 7   SPX     5472 non-null   float64
 8   XOM     5472 non-null   float64
dtypes: float64(9)
memory usage: 427.5 KB


In [220]:
close_px.tail()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2011-10-10,10.09,388.81,16.14,186.62,64.43,26.94,61.87,1194.89,76.28
2011-10-11,10.30,400.29,16.14,185.00,63.96,27.00,60.95,1195.54,76.27
2011-10-12,10.05,402.19,16.40,186.12,64.33,26.96,62.70,1207.25,77.16
2011-10-13,10.10,408.43,16.22,186.82,64.23,27.18,62.36,1203.66,76.37
2011-10-14,10.26,422.00,16.60,190.53,64.72,27.27,62.24,1224.58,78.11


In [223]:
def spx_corr(group):
    return group.corrwith(group["SPX"])
rets = close_px.pct_change().dropna()
def get_year(x):
    return x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\165094941.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = close_px.pct_change().dropna()


,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990,0.595024,0.545067,0.752187,0.738361,0.801145,0.586691,0.783168,1.0,0.517586
1991,0.453574,0.365315,0.759607,0.557046,0.646401,0.524225,0.641775,1.0,0.569335
1992,0.398180,0.498732,0.632685,0.262232,0.515740,0.492345,0.473871,1.0,0.318408
1993,0.259069,0.238578,0.447257,0.211269,0.451503,0.425377,0.385089,1.0,0.318952
1994,0.428549,0.268420,0.572996,0.385162,0.372962,0.436585,0.450516,1.0,0.395078
1995,0.291532,0.161829,0.519126,0.416390,0.315733,0.453660,0.413144,1.0,0.368752
1996,0.292344,0.191482,0.750724,0.388497,0.569232,0.564015,0.421477,1.0,0.538736
1997,0.564427,0.211435,0.827512,0.646823,0.703538,0.606171,0.509344,1.0,0.695653
1998,0.533802,0.379883,0.815243,0.623982,0.591988,0.698773,0.494213,1.0,0.369264
1999,0.099033,0.425584,0.710928,0.486167,0.517061,0.631315,0.336593,1.0,0.315383


In [225]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])
by_year.apply(corr_aapl_msft)

1990    0.408271
1991    0.266807
1992    0.450592
1993    0.236917
1994    0.361638
1995    0.258642
1996    0.147539
1997    0.196144
1998    0.364106
1999    0.329484
2000    0.275298
2001    0.563156
2002    0.571435
2003    0.486262
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [226]:
# pip install statsmodels
import statsmodels.api as sm

In [227]:
def regress(data, yvar=None, xvars=None):
    y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1
    result = sm.OLS(y, X).fit()
    return result.params

In [228]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
1990,1.512772,0.001395
1991,1.187351,0.000396
1992,1.832427,0.000164
1993,1.390470,-0.002657
1994,1.190277,0.001617
1995,0.858818,-0.001423
1996,0.829389,-0.001791
1997,0.749928,-0.001901
1998,1.164582,0.004075
1999,1.384989,0.003273


## 10.4 Group Transforms and “Unwrapped” GroupBys

In [229]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                     'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [230]:
g = df.groupby("key")["value"]
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [232]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [234]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [235]:
def times_two(group):
    return group * 2
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [236]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [237]:
def normalize(x):
    return (x-x.mean())/x.std()
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [238]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [239]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [240]:
normalized = (df["value"] - g.transform("mean")) / g.transform("std")
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64